In [ ]:
import os

import torch
import torch.nn as nn
import torch.distributed.rpc as rpc
import torch.multiprocessing as mp
import torch.nn.functional as F
import torch.distributed as dist
import torch.optim as optim
from torch import autograd

add denpendecies

In [ ]:
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '29503'

set ports

In [ ]:
rank = 0
world_size = 2
dist.init_process_group(backend='gloo', rank=rank, world_size=2)

set process groups

In [ ]:
class sendtoclient(autograd.Function):
    @staticmethod
    def forward(ctx,input):
        return input*1.0
    @staticmethod
    def backward(ctx,grad_outputs):
        print("backward,send back",grad_outputs.shape)
        dist.send(grad_outputs,1)
class NetServer(nn.Module):
    def __init__(self):
        super(NetServer, self).__init__()
        self.fc1 = nn.Linear(128*3*3, 625)
        self.fc2 = nn.Linear(625, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

build server networks

In [ ]:
net = NetServer()
learning_rate = 0.01
momentum = 0.5
optimizer = optim.SGD(net.parameters(),
                      lr=learning_rate, momentum=momentum)
print("waiting for tasks")

initialize

In [ ]:
while True:
    flag = torch.tensor(1)
    dist.recv(flag,1)
    print("for or back receive",flag)
    if flag == 0:
        
        input_data = torch.rand([128,1152]).requires_grad_()
        input_data = sendtoclient.apply(input_data)
        dist.recv(input_data,1)
        print("forward,receive",input_data.shape)
        out = torch.tensor(1.0)
        out = out.requires_grad_()
        out = net(input_data)
        
        #print(out)
        dist.send(out, 1)
        
    if flag == 1:
        
        grad_outputs = torch.rand([128,10])
        dist.recv(grad_outputs,1)
        print("backward,receive",grad_outputs.shape)
        print(grad_outputs.shape)
        grad_outputs = grad_outputs
        optimizer.zero_grad()
        out.backward(grad_outputs)
        optimizer.step()
        # dist.send(grad_i,1)